In [1]:
#These three windows are for the GeoTweetReturn results. 
#Just put your csv in the path and change the title in the third cell for the map. The rest should work automatically. 

import plotly.plotly
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#pass a CSV of tweets into df, run sentiment analysis
#Create a path to the csv and read it into a Pandas DataFrame

csv_path = 'output/holdallthetweetsB.csv'
df = pd.read_csv(csv_path, header=None)
df.columns = ["Longitude","Latitude","Tweet Text", "City&State", "Lat&Long Together"]
df.head()

#create a list to hold compound sentiments
compound_sentiments = []

# use iterrows to iterate through pandas dataframe
for index, row in df.iterrows():
    target_sample = row["Tweet Text"]
    
    # Run Vader Analysis on each tweet
    results = analyzer.polarity_scores(target_sample)

    # Run analysis
    compound = results["compound"]

    
    # Add each value to the appropriate list
    compound_sentiments.append(compound)

column_values = pd.Series(compound_sentiments)
df.insert(loc=5, column="Compound Sentiment",value=column_values)

df.head()

,Longitude,Latitude,Tweet Text,City&State,Lat&Long Together,Compound Sentiment
0,-95.234584,29.590570,Be the Beyoncé in my life or maybe be an angel...,"Pasadena, TX","[-95.2345837, 29.59057]",0.0000
1,-80.321683,25.709040,Request #1 - Beyoncé “Irreplaceable” @seanetta...,"Miami, FL","[-80.321683, 25.7090404]",0.0000
2,-95.823268,29.522325,Cardi B: Beyoncé on my stereo; \n🗣RESENTMENT🗣 ...,"Houston, TX","[-95.823268, 29.522325]",0.0000
3,-76.420402,36.786146,Justin Timberlake ft Beyoncé x until the end o...,"Portsmouth, VA","[-76.420402, 36.786146]",0.0000
4,-73.933612,40.785365,Oh how I wish I was going to #Coachella this y...,"Bronx, NY","[-73.933612, 40.785365]",0.4588


In [2]:
avgsent = df.groupby(['City&State'], as_index=True).agg(                             #get the mean sentiment for each city
                      {'Compound Sentiment':['mean']})

avgcount = df.groupby(['City&State'], as_index=True).agg(                             #get the mean sentiment for each city
                      {'Compound Sentiment':['count']})

lat = df.groupby(['City&State'], as_index=True)['City&State', 'Latitude'].head(1)         # get the lat 
lat1 = lat.set_index(['City&State'])

long = df.groupby(['City&State'], as_index=True)['City&State', 'Longitude'].head(1)     #same for long
long1 = long.set_index(['City&State'])

formap = pd.concat([avgsent, avgcount, lat1, long1], axis=1)
#latlong = pd.concat([lat1, long1],axis=1,)
formap.head()
formap = formap.rename( columns={"(tweet sentiment, mean)": "avg sentiment", "(tweet sentiment, count)": "count"})
formap = formap.reset_index()
formap.columns = ['city', 'mean sentiment', 'count', 'lat', 'long']
formap['mean sentiment'] = formap['mean sentiment'].apply(lambda x: x*100) #make the mean sentiment go from -100 to 100
formap.shape
formap.head()

,city,mean sentiment,count,lat,long
0,"Aberdeen, NC",35.660000,1,35.114539,-79.476683
1,"Addison, TX",-71.270000,2,32.932133,-96.856247
2,"Adelphia, NJ",0.000000,1,40.202463,-74.270089
3,"Akron, OH",3.963333,3,40.995137,-81.628444
4,"Alabama, USA",-6.035455,11,30.144425,-88.473228


In [3]:
#Everything should be good to go here just change the title
formap['text'] = formap['city'] + '<br>Mean Sentiment: ' + (formap['mean sentiment']).astype(str) + '<br>Tweet Count: ' + (formap['count']).astype(str)
limits = [(0,10000)]
cities = []


for i in range(len(limits)):
    
    lim = limits[i]
    formap_sub = formap[lim[0]:lim[1]]  
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = formap_sub['long'],         
        lat = formap_sub['lat'],
        text=formap_sub['text'],
        mode='markers',
        marker = dict(
            size = formap_sub['count'] * 2,
            opacity = .7,
            color = formap_sub['mean sentiment'],
            colorscale='Blackbody',
            showscale=True, 
            reversescale = False,
            colorbar= dict(title = 'Sentiment Score <br>(100 = Positive) <br>(-100 = Negative)', titleseide = 'top'),
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
    )       
    cities.append(city)

layout = dict(
        title = 'Beyoncé Sentiment (By Geo-Code)',             #change the title here. 
        showlegend = False,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
            
        ),
    )

fig = dict( data=cities, layout=layout )
plotly.offline.plot( fig, validate=False, filename='d3-bubble-map-populations.html' )

'file:///Users/jessicawilbert/Projects/Project1/Sentiment-Search-Engine/d3-bubble-map-populations.html'